## Homework 9: Text Classification with Fine-Tuned BERT

### Due: Midnight on November 5th (with 2-hour grace period) — Worth 85 points

In this final homework, we’ll explore **fine-tuning a pre-trained Transformer model (BERT)** for text classification using the **IMDB Movie Review** dataset. You’ll begin with a working baseline notebook and then conduct a series of controlled experiments to understand how data size, context length, and model architecture affect performance.

You’ll complete three problems:

* **Problem 1:** Evaluate how **sequence length** and **learning rate** jointly influence validation loss and generalization.
* **Problem 2:** Measure how **training data size** affects both model performance and total training time.
* **Problem 3:** Compare **two additional models** from the BERT family to analyze the trade-offs between model size and accuracy on this dataset.

In each problem, you’ll report your key metrics, summarize what you observed, and reflect on what you learned.

> **Note:** This homework was developed and tested on **Google Colab**, due to version conflicts when running locally. It is **strongly recommended** that you complete your work on Colab as well.

There are 6 problems, each worth 14 points, and you get one point free if you complete the entire homework.


In [1]:
# Install once per new Colab runtime
%pip -q install -U keras keras-hub tensorflow tensorflow-text datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 34.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.21.1 requires keras-hub==0.21.1, but you have keras-hub 0.23.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.19.1 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible

In [2]:

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import time
import random
import numpy as np
import keras
import keras_hub as kh
import evaluate
from datasets import load_dataset, Dataset, Features, Value, ClassLabel

from keras import mixed_precision                    # generally faster
mixed_precision.set_global_policy("mixed_float16")

### Here is where you can set global hyperparameters for this homework

In [3]:
# ---------------- Config ----------------
SEED        = 42
MAX_LEN     = 128
EPOCHS      = 3
BATCH       = 32
EVAL_BATCH  = 64
SUBSET_FRAC = 0.25   # <-- 0.25 to train and test on 25% of whole dataset during development;  set to 1.0 for full dataset

keras.utils.set_random_seed(SEED)

### Load and Preprocess the IMDB Movie Review Dataset

In [4]:
# ---- Load IMDb (raw), join train+test ----
imdb   = load_dataset("imdb")
texts  = list(imdb["train"]["text"]) + list(imdb["test"]["text"])
labels = np.array(list(imdb["train"]["label"]) + list(imdb["test"]["label"]), dtype="int32")

# ---- Build DS with explicit features (label=ClassLabel) ----
features = Features({"text": Value("string"),
                     "label": ClassLabel(num_classes=2, names=["NEG","POS"])})
all_ds = Dataset.from_dict({"text": texts, "label": labels.tolist()}, features=features)

# ---- Optional: take a stratified subset of the FULL dataset ----
if 0.0 < SUBSET_FRAC < 1.0:
    sub = all_ds.train_test_split(train_size=SUBSET_FRAC, seed=SEED, stratify_by_column="label")
    ds_pool = sub["train"]
else:
    ds_pool = all_ds

# ---- Stratified 80/10/10 split on the (possibly smaller) pool ----
# First: 80/20 train+val pool / test
splits = ds_pool.train_test_split(test_size=0.20, seed=SEED, stratify_by_column="label")
train_val_pool, test_ds = splits["train"], splits["test"]
# Then: carve 10% of full (i.e., 0.125 of the 80% pool) as validation
splits2 = train_val_pool.train_test_split(test_size=0.125, seed=SEED, stratify_by_column="label")
train_ds, val_ds = splits2["train"], splits2["test"]

# ---- Numpy arrays for Keras fit/predict ----
X_tr = np.array(train_ds["text"], dtype=object); y_tr = np.array(train_ds["label"], dtype="int32")
X_va = np.array(val_ds["text"],   dtype=object); y_va = np.array(val_ds["label"],   dtype="int32")
X_te = np.array(test_ds["text"],  dtype=object); y_te = np.array(test_ds["label"],  dtype="int32")

# ---- Quick summary ----
def _counts(ds):
    arr = np.array(ds["label"], dtype=int)
    return len(arr), np.bincount(arr, minlength=2).tolist()
print(f"Pool after SUBSET_FRAC={SUBSET_FRAC}: {len(ds_pool)} (of {len(all_ds)})")
print("Train:", _counts(train_ds), " Val:", _counts(val_ds), " Test:", _counts(test_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Pool after SUBSET_FRAC=0.25: 12500 (of 50000)
Train: (8750, [4375, 4375])  Val: (1250, [625, 625])  Test: (2500, [1250, 1250])


### Build and train a baseline Distil-Bert Text Classifier

In [5]:
# ---- Keras Hub preprocessor + classifier ----
preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
    "distil_bert_base_en_uncased", sequence_length=MAX_LEN
)
model = kh.models.DistilBertTextClassifier.from_preset(
    "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
)

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

start = time.time()

# ---- Train with early stopping (restore best val weights) ----
cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=cb,
    verbose=1,
)

# ---- Evaluate (accuracy + F1 via `evaluate`) ----
logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
y_pred = logits.argmax(axis=-1)

acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")
acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
f1  = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

# Tiny confusion matrix helper (no sklearn needed)
def confusion_matrix_np(y_true, y_pred, num_classes=2):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

print(f"\nValidation acc (best epoch): {history.history['val_acc'][np.argmin(history.history['val_loss'])]:.3f}")
print(f"\nTest accuracy: {acc:.3f}   Test F1: {f1:.3f}")
print("\nConfusion matrix:\n", confusion_matrix_np(y_te, y_pred))

end = time.time() - start
print("\nElapsed time:", time.strftime("%H:%M:%S", time.gmtime(end)))

Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 119s 245ms/step - acc: 0.7822 - loss: 0.4530 - val_acc: 0.8376 - val_loss: 0.3450
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - acc: 0.8790 - loss: 0.2900 - val_acc: 0.8592 - val_loss: 0.3400
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - acc: 0.9153 - loss: 0.2207 - val_acc: 0.8600 - val_loss: 0.3557



Validation acc (best epoch): 0.859

Test accuracy: 0.853   Test F1: 0.850

Confusion matrix:
 [[1092  158]
 [ 209 1041]]

Elapsed time: 00:02:47


# Problem 1 — Mini sweep: context length × learning rate (6 runs)

In this problem we'll see how much **context length** (`MAX_LEN`) helps, and how sensitive fine-tuning is to **learning rate**—without running a huge grid.

## Setup (keep these fixed)

* `SUBSET_FRAC = 0.25`               # use only this percentage of the whole dataset
* `EPOCHS = 3`
* `BATCH = 32` (but see note for 256 below)
* **EarlyStopping** with `restore_best_weights=True`
* Same random `SEED` for all runs
* Same data split for all runs (don’t reshuffle between runs)

### Run these 6 configurations

**For each** `MAX_LEN ∈ {128, 256, 512}`, try **two** learning rates:

* **MAX_LEN = 128**

  * `(LR = 2e-5, BATCH = 32)` – healthy default for shorter contexts.
  * `(LR = 1e-5, BATCH = 32)` – conservative LR; often a touch stabler.

* **MAX_LEN = 256**

  * `(LR = 1e-5, BATCH = 16)` – longer context → lower batch.
  * `(LR = 7.5e-6, BATCH = 16)` – even steadier if loss is noisy.

* **MAX_LEN = 512**  *(heavier quadratic attention cost)*

  * `(LR = 7.5e-6, BATCH = 8)` – safe starting point.
  * `(LR = 5e-6, BATCH = 8)` – extra caution for stability.

**If you hit an Out Of Memory error:**

* At **256** with `BATCH = 16`, drop to `BATCH = 8`.
* At **512** with `BATCH = 8`, drop to `BATCH = 4`.


Then answer the graded questions.


In [13]:
# Your code here; add as many cells as you need
def train_model(max_len, learning_rate, batch_size, name="Run"):
  "train a DistilBERT modle with specific hyperparameters"
  print(f"\n{name}: MAX_LEN={max_len}, LR={learning_rate:.1e}, BATCH={batch_size}")

  # Build preprocessor with this MAX_LEN
  preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
      "distil_bert_base_en_uncased", sequence_length=max_len
  )

  # Build model
  model = kh.models.DistilBertTextClassifier.from_preset(
      "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
  )

  # Compile with this learning rate
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
  )

  # train
  start = time.time()
  cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
  history = model.fit(
      X_tr, y_tr,
      validation_data=(X_va, y_va),
      epochs=EPOCHS,
      batch_size=batch_size,
      callbacks=cb,
      verbose=1,
  )

  # evaluate
  logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
  y_pred = logits.argmax(axis=-1)

  acc_metric = evaluate.load("accuracy")
  f1_metric  = evaluate.load("f1")
  acc = acc_metric.compute(predictions=y_pred, references=y_te)['accuracy']
  f1 = f1_metric.compute(predictions=y_pred, references=y_te)['f1']

  # get validation accuracy
  best_epoch = np.argmin(history.history['val_loss'])
  val_acc_at_best = history.history['val_acc'][best_epoch]

  elapsed = time.time() - start
  print(f"\n{name} Results:")
  print(f"Validation accuracy (best epoch): {val_acc_at_best:.4f}")
  print(f"Test accuracy: {acc:.3f}   Test F1: {f1:.3f}")
  print(f"Elapsed time: {time.strftime('H%:%M:%S', time.gmtime(elapsed))}")

  return {
      'name': name,
      'max_len' : max_len,
      'learning_rate' : learning_rate,
      'batch_size' : batch_size,
      'val_acc' : val_acc_at_best,
      'test_acc' : acc,
      'test_f1' : f1,
      'time' : elapsed
  }

results = []

# configuration 1: MAX_LEN=128, LR=2e-5, BATCH=32
results.append(train_model(128, 2e-5, 32, "Config 1: 128/2e-5/32"))

  # Configuration 2: MAX_LEN=128, LR=1e-5, BATCH=32
results.append(train_model(128, 1e-5, 32, "Config 2: 128/1e-5/32"))

# Configuration 3: MAX_LEN=256, LR=1e-5, BATCH=16
results.append(train_model(256, 1e-5, 16, "Config 3: 256/1e-5/16"))

# Configuration 4: MAX_LEN=256, LR=7.5e-6, BATCH=16
results.append(train_model(256, 7.5e-6, 16, "Config 4: 256/7.5e-6/16"))

# Configuration 5: MAX_LEN=512, LR=7.5e-6, BATCH=8
results.append(train_model(512, 7.5e-6, 8, "Config 5: 512/7.5e-6/8"))

# Configuration 6: MAX_LEN=512, LR=5e-6, BATCH=8
results.append(train_model(512, 5e-6, 8, "Config 6: 512/5e-6/8"))


# Create a summary table
import pandas as pd

df = pd.DataFrame(results)
df = df.sort_values('val_acc', ascending=False)

print("Problem 1 Results")
print(df.to_string(index=False))

# Find the best configuration
best = df.iloc[0]
print(f"\n Best Configuration:")
print(f"{best['name']}")
print(f"Validation Accuracy: {best['val_acc']:.4f}")
print(f"Test Accuracy: {best['test_acc']:.3f}")
print(f"Test F1: {best['test_f1']:.3f}")

# Set answer for grading
a1a = best['val_acc']


Config 1: 128/2e-5/32: MAX_LEN=128, LR=2.0e-05, BATCH=32
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 87s 157ms/step - acc: 0.8074 - loss: 0.4175 - val_acc: 0.8448 - val_loss: 0.3488
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - acc: 0.8909 - loss: 0.2614 - val_acc: 0.8496 - val_loss: 0.3395
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.9345 - loss: 0.1820 - val_acc: 0.8568 - val_loss: 0.3512

Config 1: 128/2e-5/32 Results:
Validation accuracy (best epoch): 0.8496
Test accuracy: 0.850   Test F1: 0.843
Elapsed time: H%:02:00

Config 2: 128/1e-5/32: MAX_LEN=128, LR=1.0e-05, BATCH=32
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 86s 155ms/step - acc: 0.7750 - loss: 0.4572 - val_acc: 0.8432 - val_loss: 0.3565
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - acc: 0.8778 - loss: 0.2933 - val_acc: 0.8520 - val_loss: 0.3460
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.9135 - loss: 0.2248 - val_acc: 0.8576 - val_loss: 0.3731

Config 2: 128/1e-5/32 Results:
Va

### Graded Questions

In [15]:
# Set a1a to the validation accuracy at min validation loss for your best configuration found in this problem

a1a = best['val_acc']             # Replace 0.0 with your answer

In [16]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a1a = {a1a:.4f}')

a1a = 0.9120


#### Question a1b:

* Does **more context** (128 → 256 → 512) consistently help?
* How much effect did the learning rate have on the validation accuracy?


#### Your Answer Here:

## Problem 2 — How much data is enough?

In this problem, you’ll investigate how model performance scales with dataset size.

**Setup.**
Use the best `MAX_LEN` and `LR` values you found in **Problem 1**.

**What to do:**

1. For each value of `SUBSET_FRAC ∈ {0.25, 0.50, 0.75, 1.00}`, train your model once and observe the displayed performance metrics.
2. Answer the discussion question below.




In [17]:
# Your code here; add as many cells as you need
def train_with_subset(subset_frac, max_len, learning_rate, batch_size, name="Run"):
    """
    Train model with a specific data subset size
    """
    print(f"{name}: SUBSET_FRAC={subset_frac}, MAX_LEN={max_len}, LR={learning_rate:.1e}")

    # recreate datasplit with new subset fractions
    imdb = load_dataset("imdb")
    texts = list(imdb["train"]["text"]) + list(imdb["test"]["text"])
    labels = np.array(list(imdb["train"]["label"]) + list(imdb["test"]["label"]), dtype="int32")

    features = Features({"text": Value("string"),
                         "label": ClassLabel(num_classes=2, names=["NEG","POS"])})
    all_ds = Dataset.from_dict({"text": texts, "label": labels.tolist()}, features=features)

    # Apply subset_frac change
    if 0.0 < subset_frac < 1.0:
        sub = all_ds.train_test_split(train_size=subset_frac, seed=SEED, stratify_by_column="label")
        ds_pool = sub["train"]
    else:
        ds_pool = all_ds

    # 80/10/10 split
    splits = ds_pool.train_test_split(test_size=0.20, seed=SEED, stratify_by_column="label")
    train_val_pool, test_ds = splits["train"], splits["test"]
    splits2 = train_val_pool.train_test_split(test_size=0.125, seed=SEED, stratify_by_column="label")
    train_ds, val_ds = splits2["train"], splits2["test"]

    # Convert to numpy
    X_tr = np.array(train_ds["text"], dtype=object)
    y_tr = np.array(train_ds["label"], dtype="int32")
    X_va = np.array(val_ds["text"], dtype=object)
    y_va = np.array(val_ds["label"], dtype="int32")
    X_te = np.array(test_ds["text"], dtype=object)
    y_te = np.array(test_ds["label"], dtype="int32")

    print(f"Data sizes - Train: {len(X_tr)}, Val: {len(X_va)}, Test: {len(X_te)}")

    # build and train model
    preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
        "distil_bert_base_en_uncased", sequence_length=max_len
    )
    model = kh.models.DistilBertTextClassifier.from_preset(
        "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

    start = time.time()
    cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]

    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=batch_size,
        callbacks=cb,
        verbose=1,
    )

    # Evaluate
    logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
    y_pred = logits.argmax(axis=-1)

    acc_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
    f1 = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

    best_epoch = np.argmin(history.history['val_loss'])
    val_acc_at_best = history.history['val_acc'][best_epoch]

    elapsed = time.time() - start

    print(f"\n{name} Results:")
    print(f"Val acc (best epoch): {val_acc_at_best:.4f}")
    print(f"Test accuracy: {acc:.3f}   Test F1: {f1:.3f}")
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(elapsed))}")

    return {
        'name': name,
        'subset_frac': subset_frac,
        'max_len': max_len,
        'lr': learning_rate,
        'batch': batch_size,
        'train_size': len(X_tr),
        'val_acc': val_acc_at_best,
        'test_acc': acc,
        'test_f1': f1,
        'time': elapsed
    }

# Setting best parameters from Problem 1
BEST_MAX_LEN = 512
BEST_LR = 7.5e-6
BEST_BATCH = 8

results_p2 = []

# Run with different data sizes
results_p2.append(train_with_subset(0.25, BEST_MAX_LEN, BEST_LR, BEST_BATCH, "25% data"))
results_p2.append(train_with_subset(0.50, BEST_MAX_LEN, BEST_LR, BEST_BATCH, "50% data"))
results_p2.append(train_with_subset(0.75, BEST_MAX_LEN, BEST_LR, BEST_BATCH, "75% data"))
results_p2.append(train_with_subset(1.00, BEST_MAX_LEN, BEST_LR, BEST_BATCH, "100% data"))

# Create summary
df_p2 = pd.DataFrame(results_p2)

print("Problem 2 Summary?")
print(df_p2.to_string(index=False))

# Find best
best_p2 = df_p2.loc[df_p2['val_acc'].idxmax()]
print(f"\n Best Configuration:")
print(f"{best_p2['name']}")
print(f"Validation Accuracy: {best_p2['val_acc']:.4f}")
print(f"Training time: {time.strftime('%H:%M:%S', time.gmtime(best_p2['time']))}")

a2a = best_p2['val_acc']


25% data: SUBSET_FRAC=0.25, MAX_LEN=512, LR=7.5e-06
Data sizes - Train: 8750, Val: 1250, Test: 2500
Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 109s 59ms/step - acc: 0.8567 - loss: 0.3226 - val_acc: 0.9152 - val_loss: 0.2220
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - acc: 0.9334 - loss: 0.1800 - val_acc: 0.9160 - val_loss: 0.2260
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - acc: 0.9590 - loss: 0.1158 - val_acc: 0.9168 - val_loss: 0.2519

25% data Results:
Val acc (best epoch): 0.9152
Test accuracy: 0.912   Test F1: 0.914
Elapsed time: 00:03:09
50% data: SUBSET_FRAC=0.5, MAX_LEN=512, LR=7.5e-06
Data sizes - Train: 17500, Val: 2500, Test: 5000
Epoch 1/3
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 153s 49ms/step - acc: 0.8854 - loss: 0.2741 - val_acc: 0.9212 - val_loss: 0.1945
Epoch 2/3
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - acc: 0.9379 - loss: 0.1684 - val_acc: 0.9160 - val_loss: 0.2029
Epoch 3/3
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - acc: 0.9633 - loss: 0.1120 

### Graded Questions

In [18]:
# Set a2a to the validation accuracy at min validation loss for your best configuration found in this problem
# (Yes, it is probably at 1.0!)

a2a = best_p2['val_acc']             # Replace 0.0 with your answer

In [19]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a2a = {a2a:.4f}')

a2a = 0.9320


#### Question a2b:

Summarize what you observed as dataset size increased. Given that validation metrics are typically reliable to only about two decimal places, do the performance gains justify using the entire dataset? What trade-offs between accuracy and computation time did you notice?

#### Your Answer Here:

# Problem 3 — Model swap: speed vs. accuracy (why: capacity matters)

In this problem we will compare encoder-only backbones of different sizes.

**Setup.** Keep the best `MAX_LEN`, `LR`, and `SUBSET_FRAC` from Problems 1–2. Only change the model/preset:

* **DistilBERT** (current baseline)
* **BERT-base** (larger/usually stronger)

**How to switch (two lines each).**

* DistilBERT:

  ```python
  preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset("distil_bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.DistilBertTextClassifier.from_preset("distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

* BERT-base:

  ```python
  preproc = kh.models.BertTextClassifierPreprocessor.from_preset("bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.BertTextClassifier.from_preset("bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

**What to do.**

1. Train/evaluate each model once with identical settings.
2. Observe the performance metrics for each.
3. Answer the graded questions.



In [27]:
# Your code here; add as many cells as you wish

# Setting best parameters from Problem 2
BEST_MAX_LEN = 512
BEST_LR = 7.5e-6
BEST_BATCH = 8
SUBSET_FRAC = 1.0

# recreate datasplit with new subset fractions
imdb = load_dataset("imdb")
texts = list(imdb["train"]["text"]) + list(imdb["test"]["text"])
labels = np.array(list(imdb["train"]["label"]) + list(imdb["test"]["label"]), dtype="int32")

features = Features({"text": Value("string"),
                      "label": ClassLabel(num_classes=2, names=["NEG","POS"])})
all_ds = Dataset.from_dict({"text": texts, "label": labels.tolist()}, features=features)

# Apply subset_frac change
subset_frac = SUBSET_FRAC

if 0.0 < subset_frac < 1.0:
    sub = all_ds.train_test_split(train_size=subset_frac, seed=SEED, stratify_by_column="label")
    ds_pool = sub["train"]
else:
    ds_pool = all_ds

# 80/10/10 split
splits = ds_pool.train_test_split(test_size=0.20, seed=SEED, stratify_by_column="label")
train_val_pool, test_ds = splits["train"], splits["test"]
splits2 = train_val_pool.train_test_split(test_size=0.125, seed=SEED, stratify_by_column="label")
train_ds, val_ds = splits2["train"], splits2["test"]

# Convert to numpy
X_tr = np.array(train_ds["text"], dtype=object)
y_tr = np.array(train_ds["label"], dtype="int32")
X_va = np.array(val_ds["text"], dtype=object)
y_va = np.array(val_ds["label"], dtype="int32")
X_te = np.array(test_ds["text"], dtype=object)
y_te = np.array(test_ds["label"], dtype="int32")

# model swap
preproc = kh.models.BertTextClassifierPreprocessor.from_preset(
    "bert_base_en_uncased", sequence_length=BEST_MAX_LEN
)
model = kh.models.BertTextClassifier.from_preset(
    "bert_base_en_uncased", num_classes=2, preprocessor=preproc
)

start = time.time()
cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=EPOCHS,
    batch_size=BEST_BATCH,
    callbacks=cb,
    verbose=1,
)

# Evaluate
# ---- Evaluate (accuracy + F1 via `evaluate`) ----
logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
y_pred = logits.argmax(axis=-1)

acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")
acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
f1  = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

# Tiny confusion matrix helper (no sklearn needed)
def confusion_matrix_np(y_true, y_pred, num_classes=2):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

print(f"\nValidation acc (best epoch): {history.history['val_sparse_categorical_accuracy'][np.argmin(history.history['val_loss'])]:.3f}")
print(f"\nTest accuracy: {acc:.3f}   Test F1: {f1:.3f}")
print("\nConfusion matrix:\n", confusion_matrix_np(y_te, y_pred))

end = time.time() - start
print("\nElapsed time:", time.strftime("%H:%M:%S", time.gmtime(end)))

Epoch 1/3
4375/4375 ━━━━━━━━━━━━━━━━━━━━ 334s 59ms/step - loss: 0.2611 - sparse_categorical_accuracy: 0.8935 - val_loss: 0.1796 - val_sparse_categorical_accuracy: 0.9348
Epoch 2/3
4375/4375 ━━━━━━━━━━━━━━━━━━━━ 254s 58ms/step - loss: 0.1617 - sparse_categorical_accuracy: 0.9395 - val_loss: 0.1991 - val_sparse_categorical_accuracy: 0.9272
Epoch 3/3
4375/4375 ━━━━━━━━━━━━━━━━━━━━ 250s 57ms/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9604 - val_loss: 0.2444 - val_sparse_categorical_accuracy: 0.9166

Validation acc (best epoch): 0.935

Test accuracy: 0.931   Test F1: 0.931

Confusion matrix:
 [[4637  363]
 [ 327 4673]]

Elapsed time: 00:14:23


In [31]:
# print all history data
history.history

bert_generation = pd.DataFrame(history.history)
bert_generation

# best validation accuarcy @ minimum loss
best_acc = bert_generation['val_sparse_categorical_accuracy'].min()
best_acc

0.9165999889373779

### Graded Questions

In [ ]:
# Set a1a to the validation accuracy at min validation loss for your best model found in this problem

a3a = 0.0             # Replace 0.0 with your answer

In [ ]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a3a = {a3a:.4f}')

a3a = 0.0000


#### Question a3b:

**Answer briefly.**

* Which model gives the best **accuracy/F1**?
* Which is **fastest** per epoch?
* Given limited development time or compute resources, which model is the best **overall choice** and why?

#### Your Answer Here: